Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.
Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.
Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred_proba.
Создайте функцию calc_pred, возвращающую предсказанный класс. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred.
Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.
Могла ли модель переобучиться? Почему?

In [66]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

задача из урока

In [105]:
# набор признаков с фиктивным
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype = np.float64)

# бинарный ответ "да" или "нет"? стОит/не стОит обращаться к этому репетитору
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

In [106]:
# функция стандартизации (загоняем в интервал 0-1)
def standard_scale(X):
    mean = X.mean(axis=0) # среднее
    std = X.std(axis=0) # стандартное отклонение
    return (X - mean) / std

In [107]:
# применяем функцию к наиболее неоднородному признаку (как в уроке)
X_st = X.copy().astype(np.float64)
X_st[:, 1:4] = standard_scale(X_st[:, 1:4]) # ептить, НЕ ВКЛЮЧАЯ 4й номер...час потратил вспоминая! курочим 1,2,3
X_st

array([[ 1.        , -0.70710678, -0.97958969, -0.89625816],
       [ 1.        , -0.70710678, -0.56713087, -0.89625816],
       [ 1.        , -0.35355339, -0.46401617,  0.38411064],
       [ 1.        ,  0.70710678, -0.77336028, -0.89625816],
       [ 1.        ,  0.        ,  0.97958969,  0.38411064],
       [ 1.        , -1.06066017, -0.36090146, -0.89625816],
       [ 1.        ,  0.70710678,  1.08270439,  1.66447944],
       [ 1.        ,  2.47487373,  2.11385144,  1.66447944],
       [ 1.        , -0.70710678, -1.08270439, -0.89625816],
       [ 1.        , -0.35355339,  0.05155735,  0.38411064]])

**1. *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.**

Решение мной откровенно сдуто из разбора, но метод я покурил и он мне кажется более логичным и точным, чем второй. Здесь мы немного "подкручиваем" неудобные для нас значения: на работу модели это изменение числа добавлением 1 в -8 степени толком не повлияет, но ошибку исключит. Интересная штука, запомнил.

In [79]:
def calc_logloss(y, y_pred):
    y_pred = np.where(y_pred == 1, 1 - 1e-8, y_pred)
    y_pred = np.where(y_pred == 0, 0 + 1e-8, y_pred)
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [80]:
eps = 1e-8

np.clip([1, 0.5, 0], a_min=eps, a_max=1 - eps)

array([9.9999999e-01, 5.0000000e-01, 1.0000000e-08])

In [81]:
def calc_logloss(y, y_pred):
    eps = 1e-8
    y_pred = np.clip(y_pred, eps, 1 - eps)
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

**2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.**

In [82]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [83]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
            
    final_error = calc_logloss(y, y_pred)
    return W, final_error

2 сделано руками. 10к итераций, скорость 5

In [92]:
W = eval_model(X_st, y, iterations=1000, eta=5)

0 [ 0.31921832 -1.49236188 -0.51944502  0.90063097] 0.7609587975918891
100 [ 1.02648231 -4.54832431 -3.59179908  9.30959481] 0.20250464939796467
200 [ 1.6892199  -5.95938735 -5.24377617 12.8889147 ] 0.1660216906956335
300 [ 2.16427217 -7.03257372 -6.46131761 15.57316506] 0.14571407985869567
400 [ 2.53489717 -7.89919932 -7.43532663 17.73437349] 0.13262029459685326
500 [ 2.83993464 -8.62762653 -8.25428116 19.55431928] 0.12336720436123445
600 [ 3.10044196 -9.25773905 -8.9669849  21.13604785] 0.11639485010009273
700 [ 3.32896268 -9.81457332 -9.60297039 22.54308371] 0.11088757078464562
800 [  3.53350926 -10.31476145 -10.18133992  23.81708871] 0.10637887593552017
900 [  3.7194846  -10.76986477 -10.71509821  24.9867388 ] 0.10258293501565244


**3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).**

In [103]:
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(np.dot(X, W))
    return y_pred_proba

In [104]:
calc_pred_proba(W, X)

ValueError: shapes (10,4) and (2,) not aligned: 4 (dim 1) != 2 (dim 0)

**4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).**

In [101]:
def calc_pred(W, X_st):
    y_pred_proba = calc_pred_proba(W, X_st)
    y_pred = np.where(y_pred_proba > 0.5, 1, 0)
    return y_pred

In [102]:
pred = calc_pred(W, X)
prediction

ValueError: shapes (10,4) and (2,) not aligned: 4 (dim 1) != 2 (dim 0)